In [1]:
import JLD2
using Discretizers
using CSV
using Seaborn

import Base.Filesystem: joinpath

In [2]:
include("src\\NGSIM.jl")

gen_distances (generic function with 1 method)

In [3]:
traj_file = Base.Filesystem.basename(NGSIM.TRAJDATA_PATHS[1])
traj_file = traj_file[1:search(traj_file, '.')-1]

DATA_PATH = "../data/" * traj_file

"../data/trajdata_i101_trajectories-0750am-0805am"

In [43]:
!isdir(DATA_PATH) && Base.Filesystem.mkdir(DATA_PATH)

In [4]:
JLD2.@load "$(DATA_PATH)/td.jld" td
rd = NGSIM.ROADWAY_101
(TS, id_lookup) = td_sparse(td)
;

In [58]:
fs = [
    s::VehicleState -> s.v * cos(s.posF.ϕ), 
    s::VehicleState -> s.v * sin(s.posF.ϕ),
    s::VehicleState -> length(rd[s.posF.roadind.tag.segment].lanes) - s.posF.roadind.tag.lane + 1
]

fs_enum = enumerate(fs)
X = AF64(nnz(TS), length(fs))
;

In [59]:
@inbounds for (t, s) in TS.nzval |> enumerate
    for (i, f) in fs_enum
        X[t, i] = f(s)
    end
end

XΓ = X[:, 1:2]
XΔ = round.(Int, X[:, 3])
;

In [60]:
disc1 = LinearDiscretizer(binedges(DiscretizeUniformWidth(150), XΓ[:, 1]))
c1 = get_discretization_counts(disc1, XΓ[:, 1])
;

In [61]:
disc2 = LinearDiscretizer(binedges(DiscretizeUniformWidth(30), XΓ[:, 2]))
c2 = get_discretization_counts(disc2, XΓ[:, 2])
;

In [62]:
enc1 = encode(disc1, XΓ[:, 1])
enc2 = encode(disc1, XΓ[:, 2])
D = [enc1 enc2 XΔ]
;

In [64]:
writecsv(joinpath(DATA_PATH, "X_disc.csv"), D)

In [65]:
writecsv(joinpath(DATA_PATH, "X1_bins.csv"), disc1.binedges)

In [66]:
writecsv(joinpath(DATA_PATH, "X2_bins.csv"), disc2.binedges)

In [49]:
writecsv(joinpath(DATA_PATH, "X.csv"), X)

In [51]:
writecsv(joinpath(DATA_PATH, "traj_lengths.csv"), diff(TS.colptr))